## Бонусное задание: word2vec на PyTorch

Как вы уже могли заметить, идея, лежащая в основе [word2vec](https://arxiv.org/pdf/1310.4546), достаточно общая. В данном задании вы реализуете его самостоятельно.

Дисклеймер: не стоит удивляться тому, что реализация от `gensim` (или аналоги) обучается быстрее и работает точнее. Она использует множество доработок и ускорений, а также достаточно эффективный код. Ваша задача добиться промежуточных результатов за разумное время.

P.s. Как ни странно, GPU в этом задании нам не потребуется.

__Requirements:__ if you're running locally, in the selected environment run the following command:

```pip install --upgrade nltk bokeh umap-learn```


In [1]:
!pip install ipykernel -U --user --force-reinstall

  Using cached ipykernel-6.29.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached appnope-0.1.4-py2.py3-none-any.whl.metadata (908 bytes)
  Using cached comm-0.2.2-py3-none-any.whl.metadata (3.7 kB)
  Using cached debugpy-1.8.7-py2.py3-none-any.whl.metadata (1.1 kB)
  Using cached ipython-8.29.0-py3-none-any.whl.metadata (5.0 kB)
  Using cached jupyter_client-8.6.3-py3-none-any.whl.metadata (8.3 kB)
  Using cached jupyter_core-5.7.2-py3-none-any.whl.metadata (3.4 kB)
  Using cached matplotlib_inline-0.1.7-py3-none-any.whl.metadata (3.9 kB)
  Using cached nest_asyncio-1.6.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached packaging-24.1-py3-none-any.whl.metadata (3.2 kB)
  Using cached psutil-6.1.0-cp36-abi3-macosx_11_0_arm64.whl.metadata (22 kB)
  Using cached pyzmq-26.2.0-cp310-cp310-macosx_10_15_universal2.whl.metadata (6.2 kB)
  Using cached tornado-6.4.1-cp38-abi3-macosx_10_9_universal2.whl.metadata (2.5 kB)
  Using cached traitlets-5.14.3-py3-none-any.whl.metadata (10 kB)
  Using

In [1]:
!pip install --upgrade nltk bokeh umap-learn

  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached bokeh-3.6.0-py3-none-any.whl.metadata (12 kB)
  Using cached umap_learn-0.5.6-py3-none-any.whl.metadata (21 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached xyzservices-2024.9.0-py3-none-any.whl.metadata (4.1 kB)
  Using cached pynndescent-0.5.13-py3-none-any.whl.metadata (6.8 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached bokeh-3.6.0-py3-none-any.whl (6.9 MB)
Using cached umap_learn-0.5.6-py3-none-any.whl (85 kB)
Using cached jinja2-3.1.4-py3-none-any.whl (133 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 1.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 814.5 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 661.8 kB/s eta 0:00:0000:0100:01
   ━━━━━━━

In [2]:
!pip install matplotlib


  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.0-py3-none-any.whl.metadata (5.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 1.0 MB/s eta 0:00:000:00:0100:01
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 739.1 kB/s eta 0:00:00a 0:00:01
Using cached pyparsing-3.2.0-py3-none-any.whl (106 kB)


In [4]:
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 MB 910.8 kB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 887.2 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 631.9 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 1.8 MB/s eta 0:00:00-:--:--


In [5]:
import itertools
import random
import string
from collections import Counter
from itertools import chain

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import umap
from IPython.display import clear_output
from matplotlib import pyplot as plt
from nltk.tokenize import WordPunctTokenizer
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR
from tqdm.auto import tqdm as tqdma

/Users/vera/miniconda3/envs/my_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt -nc
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

zsh:1: no matches found: https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1


In [8]:
data = list(open("./quora.txt", encoding="utf-8"))
data[50]

"What TV shows or books help you read people's body language?\n"

Токенизация – первый шаг.
Тексты, с которыми мы работаем, включают в себя пунктуацию, смайлики и прочие нестандартные токены, так что простой `str.split` не подойдет.

Обратимся к `nltk` - библиотеку, нашла широкое применеие в области NLP.

In [9]:
tokenizer = WordPunctTokenizer()

#Токенизатор разбивает каждую строку на отдельные слова (токены).
print(tokenizer.tokenize(data[50])) 


['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [10]:
data_tok = [
    tokenizer.tokenize(
        line.translate(str.maketrans("", "", string.punctuation)).lower() 
        #Для каждой строки в data метод translate удаляет все знаки пунктуации, а метод lower преобразует все символы в нижний регистр.
    )
    for line in data
]

#Создается новый список data_tok, в который входят только те токенизированные строки, которые содержат 3 или более токенов (слов). 
# Строки с числом токенов меньше 3 отбрасываются.
data_tok = [x for x in data_tok if len(x) >= 3]


Несколько проверок:

In [11]:
#Проверка, что каждый элемент в data_tok является списком или кортежем
assert all(
    isinstance(row, (list, tuple)) for row in data_tok
), "please convert each line into a list of tokens (strings)"

#Проверка, что каждый элемент внутри строк в data_tok является строкой
assert all(
    all(isinstance(tok, str) for tok in row) for row in data_tok
), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all("a" <= x.lower() <= "z" for x in tok) #Функция is_latin используется для проверки, содержит ли токен только латинские символы (буквы от 'a' до 'z' без учёта регистра).

#Проверка, что строки с латинскими символами — в нижнем регистре
assert all(
    map(lambda l: not is_latin(l) or l.islower(), map(" ".join, data_tok))
), "please make sure to lowercase the data"

Ниже заданы константы ширины окна контекста и проведена предобработка для построения skip-gram модели.

In [12]:
min_count = 5
window_radius = 5

In [13]:
#chain.from_iterable(data_tok) разворачивает все списки из data_tok (где data_tok — список списков токенов) в один плоский список слов.
#Counter подсчитывает частоту каждого слова, создавая словарь, в котором ключами являются слова, а значениями — их частоты.
vocabulary_with_counter = Counter(chain.from_iterable(data_tok))

word_count_dict = dict()
for word, counter in vocabulary_with_counter.items():
    if counter >= min_count:
        word_count_dict[word] = counter

vocabulary = set(word_count_dict.keys())

#удаляется для освобождения памяти
del vocabulary_with_counter

In [14]:
#Этот словарь word_to_index сопоставляет каждому слову из vocabulary уникальный числовой индекс.
#enumerate(vocabulary) перебирает слова из vocabulary, назначая каждому слову порядковый номер (index), начиная с 0.
word_to_index = {word: index for index, word in enumerate(vocabulary)}

#Этот словарь index_to_word является обратным к word_to_index, то есть каждому индексу сопоставляется его слово.
#Он создается путем перебора элементов word_to_index, где ключ (word) становится значением, а значение (index) — ключом.
index_to_word = {index: word for word, index in word_to_index.items()}

Пары `(слово, контекст)` на основе доступного датасета сгенерированы ниже.

In [15]:
context_pairs = []

for text in data_tok:
    #Для каждого слова в предложении (в переменной text) определяет индекс i и само слово central_word, которое будет считаться центральным для текущего окна.
    for i, central_word in enumerate(text):
        #Определяет индексы слов, находящихся в пределах радиуса окна window_radius вокруг central_word.
        context_indices = range(
            max(0, i - window_radius), min(i + window_radius, len(text)) #гарантирует, что диапазон не выходит за начало текста и конец
        )
        #Перебирает все индексы соседних слов в текущем окне
        for j in context_indices:
            #Пропускает итерацию, если текущий индекс совпадает с индексом центрального слова, чтобы не создавать пару, состоящую из слова и самого себя.
            if j == i:
                continue
            context_word = text[j]
            if central_word in vocabulary and context_word in vocabulary:
                context_pairs.append(
                    #Если оба слова есть в словаре, создает пару, используя индексы word_to_index для центрального и контекстного слов, и добавляет ее в список context_pairs.
                    (word_to_index[central_word], word_to_index[context_word])
                )
#Таким образом, в итоге создается список context_pairs, 
# содержащий пары (индекс целевого слова, индекс контекстного слова), 
# которые будут использоваться для обучения Skip-gram модели.

print(f"Generated {len(context_pairs)} pairs of target and context words.")

Generated 40220313 pairs of target and context words.


#### Подзадача №1: subsampling
Для того, чтобы сгладить разницу в частоте встречаемсости слов, необходимо реализовать механизм subsampling'а.
Для этого вам необходимо реализовать функцию ниже.

Вероятность **исключить** слово из обучения (на фиксированном шаге) вычисляется как
$$
P_\text{drop}(w_i)=1 - \sqrt{\frac{t}{f(w_i)}},
$$
где $f(w_i)$ – нормированная частота встречаемости слова, а $t$ – заданный порог (threshold).

In [18]:
def subsample_frequent_words(word_count_dict, threshold=1e-5):
    """
    Calculates the subsampling probabilities for words based on their frequencies.

    This function is used to determine the probability of keeping a word in the dataset
    when subsampling frequent words. The method used is inspired by the subsampling approach
    in Word2Vec, where each word's frequency affects its probability of being kept.

    Parameters:
    - word_count_dict (dict): A dictionary where keys are words and values are the counts of those words.
    - threshold (float, optional): A threshold parameter used to adjust the frequency of word subsampling.
                                   Defaults to 1e-5.

    Returns:
    - dict: A dictionary where keys are words and values are the probabilities of keeping each word.

    Example:
    >>> word_counts = {'the': 5000, 'is': 1000, 'apple': 50}
    >>> subsample_frequent_words(word_counts)
    {'the': 0.028, 'is': 0.223, 'apple': 1.0}
    """

    ### YOUR CODE HERE

    # Общее количество слов в словаре
    total_count = sum(word_count_dict.values())
    
    # Нормализуем частоты слов, чтобы получить вероятности
    # В результате, каждая пара "слово — частота" из word_count_dict превращается в пару "слово — вероятность" в новом словаре word_freq.
    word_freq = {word: count / total_count for word, count in word_count_dict.items()} #word_count_dict.items() возвращает набор всех пар "ключ — значение" из исходного словаря word_count_dict

    #Этот код создаёт словарь keep_prob_dict, где каждому слову из word_freq присваивается вероятность его сохранения, зависящая от порогового значения threshold и частоты слова freq.
    keep_prob_dict = {
        word: max(0,1 - (threshold / freq) ** 0.5) if freq > threshold else 1.0
        for word, freq in word_freq.items()
    }  # THIS IS A PLACEHOLDER!
    return keep_prob_dict

#### Подзадача №2: negative sampling
Для более эффективного обучения необходимо не только предсказывать высокие вероятности для слов из контекста, но и предсказывать низкие для слов, не встреченных в контексте. Для этого вам необходимо вычислить вероятност использовать слово в качестве negative sample, реализовав функцию ниже.

В оригинальной статье предлагается оценивать вероятность слов выступать в качестве negative sample согласно распределению $P_n(w)$
$$
P_n(w) = \frac{U(w)^{3/4}}{Z},
$$

где $U(w)$ распределение слов по частоте (или, как его еще называют, по униграммам), а $Z$ – нормировочная константа, чтобы общая мера была равна $1$.

In [19]:
def get_negative_sampling_prob(word_count_dict):
    """
    Calculates the negative sampling probabilities for words based on their frequencies.

    This function adjusts the frequency of each word raised to the power of 0.75, which is
    commonly used in algorithms like Word2Vec to moderate the influence of very frequent words.
    It then normalizes these adjusted frequencies to ensure they sum to 1, forming a probability
    distribution used for negative sampling.

    Parameters:
    - word_councc  c c t_dict (dict): A dictionary where keys are words and values are the counts of those words.

    Returns:
    - dict: A dictionary where keys are words and values are the probabilities of selecting each word
            for negative sampling.

    Example:
    >>> word_counts = {'the': 5000, 'is': 1000, 'apple': 50}
    >>> get_negative_sampling_prob(word_counts)
    {'the': 0.298, 'is': 0.160, 'apple': 0.042}
    """

    ### YOUR CODE HERE
    
   # Вычисляем частоты слов с учетом степени 3/4
    adjusted_freq = {word: count ** (3/4) for word, count in word_count_dict.items()}
    
    # Нормируем частоты, чтобы сумма была равна 1
    total_adjusted_freq = sum(adjusted_freq.values())
    
    negative_sampling_prob_dict = {
        word: freq / total_adjusted_freq for word, freq in adjusted_freq.items()
    } # THIS IS A PLACEHOLDER!
    return negative_sampling_prob_dict

Для удобства, преобразуем полученные словари в массивы (т.к. все слова все равно уже пронумерованы).

In [20]:
# Иными словами, это выражение проверяет, 
# что после выполнения функции subsample_frequent_words каждый ключ (то есть, каждое слово) из исходного словаря word_count_dict присутствует и в выходном словаре keep_prob_dict.

keep_prob_dict = subsample_frequent_words(word_count_dict)

#assert — это оператор, который проверяет истинность утверждения, указанного справа от него. Если утверждение неверно, программа остановится и выдаст ошибку AssertionError.
assert keep_prob_dict.keys() == word_count_dict.keys()

In [21]:
negative_sampling_prob_dict = get_negative_sampling_prob(word_count_dict)
assert negative_sampling_prob_dict.keys() == negative_sampling_prob_dict.keys()

# Этот код проверяет, что сумма всех значений в negative_sampling_prob_dict приблизительно равна 
# Функция np.allclose из библиотеки numpy проверяет, что два числа (или массивы) близки друг к другу с учётом допустимой погрешности. 
# Это полезно в случае работы с числами с плавающей точкой, так как такие вычисления могут давать небольшие отклонения от ожидаемого значения из-за особенностей округления.
assert np.allclose(sum(negative_sampling_prob_dict.values()), 1)

# Такое утверждение проверяет, что словарь negative_sampling_prob_dict представляет корректное распределение вероятностей, где сумма всех вероятностей должна быть равна 1. 
# Это особенно важно в задачах, где требуется выборка с заданной вероятностью для каждой категории (в данном случае, слов), 
# например, для метода негативной выборки (negative sampling) в обучении моделей обработки текста.

In [22]:
keep_prob_array = np.array(
    [keep_prob_dict[index_to_word[idx]] for idx in range(len(word_to_index))]
)
negative_sampling_prob_array = np.array(
    [
        negative_sampling_prob_dict[index_to_word[idx]]
        for idx in range(len(word_to_index))
    ]
)

Если все прошло успешно, функция ниже поможет вам с генерацией подвыборок (батчей).

### Зачем нужен батч?
Вместо того, чтобы обучать модель на всех данных сразу, что может быть медленно и требовательно к памяти, данные разделяют на батчи. Модель обновляет свои параметры после обработки каждого батча, а не всей выборки, что ускоряет обучение и снижает нагрузку на память.

### Преимущества использования батчей
*Снижение требований к памяти:* Батчи позволяют работать с частями данных, вместо всей обучающей выборки, что особенно полезно для больших наборов данных.

*Ускорение обучения:* Параметры модели обновляются чаще (после каждого батча), что способствует более быстрой сходимости модели.

*Улучшение обобщающей способности:* Поскольку данные разбиваются на небольшие подмножества, градиенты, вычисленные для каждого батча, имеют случайные отклонения, которые могут помочь модели избежать локальных минимумов.

In [23]:
# Функция generate_batch_with_neg_samples создаёт обучающую выборку для моделей на основе метода негативной выборки. 
# Она формирует батч из пар "центр-контекст" (слово и его сосед) и для каждой пары добавляет несколько негативных примеров:
# context_pairs: список пар "центр-контекст" (center, context), где center — центральное слово, а context — соседнее слово.
# batch_size: желаемый размер батча.
# keep_prob_array: массив вероятностей сохранения для каждого слова в словаре. Вероятность указана для индекса слова, и она используется для субсемплинга частых слов.
# word_to_index: словарь, отображающий слова на их индексы в массиве.
# num_negatives: количество негативных примеров, которое нужно сгенерировать для каждого положительного примера.
# negative_sampling_prob_array: массив, содержащий вероятности, с которыми каждое слово должно быть выбрано в качестве негативного примера. Сумма всех вероятностей должна быть 1. 

def generate_batch_with_neg_samples(
    context_pairs,
    batch_size,
    keep_prob_array,
    word_to_index,
    num_negatives,
    negative_sampling_prob_array,
):
    # Здесь batch хранит пары "центр-контекст", а neg_samples — негативные примеры, соответствующие каждой паре.
    batch = []
    neg_samples = []

    # Цикл продолжается, пока размер batch не достигнет batch_size.
    while len(batch) < batch_size:
        center, context = random.choice(context_pairs)
        # Для каждого центрального слова (center) проводится случайный тест на его сохранение, основанный на вероятности из keep_prob_array. 
        # Если случайное число меньше этой вероятности, пара "центр-контекст" добавляется в batch.
        if random.random() < keep_prob_array[center]:
            batch.append((center, context))
            # Здесь np.random.choice выбирает num_negatives слов на основе массива negative_sampling_prob_array, который содержит вероятности для каждого слова. 
            # Эти негативные примеры добавляются в neg_samples.
            neg_sample = np.random.choice(
                range(len(negative_sampling_prob_array)),
                size=num_negatives,
                p=negative_sampling_prob_array,
            )
            neg_samples.append(neg_sample)
    # Пары "центр-контекст" и негативные примеры превращаются в массивы, которые затем возвращаются.
    batch = np.array(batch)
    neg_samples = np.vstack(neg_samples)
    return batch, neg_samples

In [24]:
batch_size = 4 # это количество пар "центр-контекст" (положительных примеров), которые будут включены в батч.
num_negatives = 15 # это количество негативных примеров, которое будет сгенерировано для каждой пары "центр-контекст" в батче.
batch, neg_samples = generate_batch_with_neg_samples(
    context_pairs,
    batch_size,
    keep_prob_array,
    word_to_index,
    num_negatives,
    negative_sampling_prob_array,
)

Наконец, время реализовать модель. Обращаем ваше внимание, использование линейных слоев (`nn.Linear`) далеко не всегда оправданно!

Напомним, что в случае negative sampling решается задача максимизации следующего функционала:

$$
\mathcal{L} = \log \sigma({\mathbf{v}'_{w_O}}^\top \mathbf{v}_{w_I}) + \sum_{i=1}^{k} \mathbb{E}_{w_i \sim P_n(w)} \left[ \log \sigma({-\mathbf{v}'_{w_i}}^\top \mathbf{v}_{w_I}) \right],
$$

где:
- $\mathbf{v}_{w_I}$ – вектор центрального слова $w_I$,
- $\mathbf{v}'_{w_O}$ – вектор слова из контекста $w_O$,
- $k$ – число negative samplesЮ,
- $P_n(w)$ – распределение negative samples, заданное выше,
- $\sigma$ – сигмоида.

In [120]:
# Этот код определяет класс SkipGramModelWithNegSampling, который реализует модель Skip-gram с негативным сэмплированием, использующую библиотеку PyTorch. 
# Модель используется для обучения векторных представлений слов (эмбеддингов) с помощью метода Skip-gram, где цель состоит в том,
#  чтобы предсказать контекстные слова (слова, окружающие центральное слово) из центральных слов.
class SkipGramModelWithNegSampling(nn.Module):
    # vocab_size: размер словаря, то есть общее количество уникальных слов, которые будут использоваться в модели.
    # embedding_dim: размерность эмбеддингов (векторных представлений) для каждого слова.
    # nn.Embedding: это слой PyTorch, который создает матрицу эмбеддингов, где строки соответствуют индексам слов, а столбцы — значениям векторов.
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        # Инициализируем эмбеддинги для центральных и контекстных слов
        self.center_embeddings = nn.Embedding(vocab_size, embedding_dim)  # Эмбеддинги центральных слов
        self.context_embeddings = nn.Embedding(vocab_size, embedding_dim)  # Эмбеддинги контекстных слов

    # Функция forward принимает индексы центральных и контекстных слов и вычисляет положительные и негативные оценки для этих слов. 
    # Эти оценки позже могут быть использованы для вычисления потерь в процессе обучения, что позволяет модели улучшать качество эмбеддингов, делая их более информативными для задачи предсказания. 
    
    # center_words: индексы центральных слов в батче.
    # pos_context_words: индексы положительных контекстных слов (правильные контексты для центральных слов).
    # neg_context_words: индексы негативных контекстных слов (случайные слова, которые не являются контекстами для центральных слов).
    # center_vectors: векторные представления для центральных слов.
    # pos_context_vectors: векторные представления для положительных контекстных слов.
    # neg_context_vectors: векторные представления для негативных контекстных слов.
    def forward(self, center_words, pos_context_words, neg_context_words):

        center_vectors = self.center_embeddings(center_words)  # [batch_size, embedding_dim]
        pos_context_vectors = self.context_embeddings(pos_context_words)  # [batch_size, embedding_dim]
        neg_context_vectors = self.context_embeddings(neg_context_words)  # [batch_size * num_negatives, embedding_dim]
        
       # Calculate positive scores (should be [batch_size], где каждый элемент представляет собой оценку для соответствующей пары "центр-контекст)
       # Оценка положительных пар "центр-контекст" вычисляется путем поэлементного умножения векторов центральных и контекстных слов, а затем суммирования по размерности векторов (размерность 1).
        pos_scores = (center_vectors * pos_context_vectors).sum(dim=1)  

        # Calculate negative scores (should be [batch_size, num_negatives], где каждый элемент в строке соответствует оценкам для негативных примеров, связанных с данным центральным словом.)
        # center_vectors.unsqueeze(1) добавляет новую размерность, превращая тензор размера [batch_size, embedding_dim] в [batch_size, 1, embedding_dim].
        # Затем neg_context_vectors.view(center_vectors.size(0), -1, center_vectors.size(1)) изменяет размерность негативных векторов, чтобы они соответствовали размерности центра.
        # После этого выполняется поэлементное умножение и суммирование по размерности векторов (размерность 2).
        neg_scores = (center_vectors.unsqueeze(1) * neg_context_vectors.view(center_vectors.size(0), -1, center_vectors.size(1))).sum(dim=2)  # Shape: [batch_size, num_negatives]

        # Return scores, ensuring they are in the expected shape


        return pos_scores, neg_scores

### Зачем это нужно?

*Переключение между устройствами:* В PyTorch часто требуется выбирать между CPU и GPU в зависимости от доступных ресурсов и требований к производительности. Если ваш код будет выполняться на GPU, вы могли бы указать "cuda" вместо "cpu".

*Кроссплатформенность:* Использование torch.device позволяет писать код, который легко адаптируется для выполнения на разных устройствах. Например, вы можете проверять доступность GPU и, если он доступен, использовать его

In [122]:
# torch.device: Это класс в PyTorch, который представляет устройство, на котором будут размещены тензоры и выполняются операции. 
# Он позволяет вам легко переключаться между различными устройствами, такими как CPU и GPU.
# В данном случае, строка "cpu" указывает, что устройство, на котором будут выполняться операции, — это центральный процессор (CPU).
# Это значит, что все тензоры и вычисления, которые используют это устройство, будут выполняться на процессоре вашего компьютера, а не на графическом процессоре (GPU).
device = torch.device("cpu")

In [123]:
# В результате выполнения этих строк кода вы создаете модель Skip-Gram с негативной выборкой, настраиваете оптимизатор для обновления параметров модели, 
# устанавливаете планировщик скорости обучения для оптимизации процесса обучения и определяете функцию потерь, которая будет использоваться для оценки эффективности модели. 
# Все это является важными этапами подготовки к обучению нейронной сети.


vocab_size = len(word_to_index)
embedding_dim = 32 #Указывает размерность векторных представлений слов (эмбеддингов). В данном случае размерность равна 32, что означает, что каждое слово будет представлено вектором размером 32.
num_negatives = 15 #Указывает количество негативных выборок, которые будут использоваться в процессе обучения модели. В данном случае вы выбираете 15 негативных примеров для каждого положительного примера.

model = SkipGramModelWithNegSampling(vocab_size, embedding_dim).to(device)

# model.parameters(): Получает параметры модели (включая эмбеддинги) для оптимизации.
# lr=0.05: Устанавливает скорость обучения (learning rate) для оптимизатора. Скорость обучения контролирует, насколько сильно обновляются параметры на каждой итерации.
optimizer = optim.Adam(model.parameters(), lr=0.05)

# ReduceLROnPlateau: Инициализация планировщика изменения скорости обучения, который уменьшает скорость обучения, если модель не показывает улучшение в течение определенного количества эпох.
# optimizer: Передает оптимизатор, который будет изменяться.
# factor=0.5: Указывает, что скорость обучения будет уменьшаться в 2 раза, когда срабатывает условие (то есть когда нет улучшения).
# patience=150: Определяет, сколько эпох модель может обучаться без улучшения, прежде чем сработает уменьшение скорости обучения.
lr_scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=150)

# Инициализация функции потерь для бинарной классификации с логитами. 
# Эта функция объединяет два этапа: применение сигмоиды и расчет бинарной кросс-энтропии. Она будет использоваться для оценки качества предсказаний модели во время обучения.
criterion = nn.BCEWithLogitsLoss()

In [124]:
# В приведенном вами коде происходит подсчет общего числа параметров в модели Skip-Gram с негативной выборкой, а затем выполняется проверка, 
# соответствует ли это значение ожидаемому количеству параметров на основе размера словаря и размерности эмбеддингов.

params_counter = 0
for weights in model.parameters():
    params_counter += weights.shape.numel()
assert params_counter == len(word_to_index) * embedding_dim * 2

In [125]:
# model: модель Skip-Gram с негативной выборкой.
# context_pairs: список пар центральных и контекстных слов.
# keep_prob_array: массив вероятностей, используемых для отсева слов.
# word_to_index: словарь, сопоставляющий слова с их индексами.
# batch_size: размер батча.
# num_negatives: количество негативных образцов для выборки.
# negative_sampling_prob_array: вероятности выборки негативных слов.
# steps: количество шагов обучения.
# optimizer: оптимизатор для обновления параметров модели (по умолчанию optimizer).
# lr_scheduler: планировщик для изменения скорости обучения (по умолчанию lr_scheduler).
# device: устройство для выполнения (например, CPU или GPU).

def train_skipgram_with_neg_sampling(
    model,
    context_pairs,
    keep_prob_array,
    word_to_index,
    batch_size,
    num_negatives,
    negative_sampling_prob_array,
    steps,
    optimizer=optimizer,
    lr_scheduler=lr_scheduler,
    device=device,
):
    # создается вектор меток для положительных примеров (все значения равны 1, так как это положительные примеры).
    pos_labels = torch.ones(batch_size).to(device)
    # создается вектор меток для негативных примеров (все значения равны 0).
    neg_labels = torch.zeros(batch_size, num_negatives).to(device)
    # список для хранения значений потерь на каждом шаге обучения.
    loss_history = []

    for step in tqdma(range(steps)):
        batch, neg_samples = generate_batch_with_neg_samples(
            context_pairs,
            batch_size,
            keep_prob_array,
            word_to_index,
            num_negatives,
            negative_sampling_prob_array,
        )

        # Создание тензоров: центральные слова, положительные и негативные контекстные слова преобразуются в тензоры и отправляются на нужное устройство (device).
        center_words = torch.tensor([pair[0] for pair in batch], dtype=torch.long).to(
            device
        )
        pos_context_words = torch.tensor(
            [pair[1] for pair in batch], dtype=torch.long
        ).to(device)
        neg_context_words = torch.tensor(neg_samples, dtype=torch.long).to(device)

        # Обнуление градиентов: обнуление накопленных градиентов перед обратным проходом.
        optimizer.zero_grad()
        # Вычисление оценок: вызов модели для получения оценок положительных и негативных пар слов.
        pos_scores, neg_scores = model(
            center_words, pos_context_words, neg_context_words
        )

        # Вычисление потерь
        loss_pos = criterion(pos_scores, pos_labels)
        loss_neg = criterion(neg_scores, neg_labels)
        loss = loss_pos + loss_neg

        # Обратное распространение ошибки: вычисление градиентов по отношению к параметрам модели.
        # Обновление параметров: оптимизатор обновляет параметры модели на основе вычисленных градиентов.
        loss.backward()
        optimizer.step()

        loss_history.append(loss.item())
        # Обновление скорости обучения: планировщик скорости обучения (lr_scheduler) обновляет скорость обучения на основе последних значений потерь.
        lr_scheduler.step(loss_history[-1])

        # каждые 100 шагов обучения выводится текущее значение шага, среднее значение потерь за последние 100 шагов и текущая скорость обучения.
        if step % 100 == 0:
            print(
                f"Step {step}, Loss: {np.mean(loss_history[-100:])}, learning rate: {lr_scheduler._last_lr}"
            )

In [126]:
steps = 2500
batch_size = 512
train_skipgram_with_neg_sampling(
    model,
    context_pairs,
    keep_prob_array,
    word_to_index,
    batch_size,
    num_negatives,
    negative_sampling_prob_array,
    steps,
)

  0%|          | 1/2500 [00:00<29:14,  1.42it/s]

Step 0, Loss: 4.6023783683776855, learning rate: [0.05]


  4%|▍         | 101/2500 [01:07<26:34,  1.50it/s]

Step 100, Loss: 2.6979678475856783, learning rate: [0.05]


  8%|▊         | 201/2500 [02:12<25:11,  1.52it/s]

...

 92%|█████████▏| 2301/2500 [25:10<02:09,  1.53it/s]

Step 2300, Loss: 1.4428039014339447, learning rate: [0.000390625]


 96%|█████████▌| 2401/2500 [26:15<01:04,  1.53it/s]

Step 2400, Loss: 1.4427489888668061, learning rate: [0.0001953125]


100%|██████████| 2500/2500 [27:20<00:00,  1.52it/s]


Наконец, используйте полученную матрицу весов в качестве матрицы в векторными представлениями слов. Рекомендуем использовать для сдачи матрицу, которая отвечала за слова из контекста (т.е. декодера).

In [127]:
# В PyTorch у моделей есть параметры, такие как веса и смещения слоев, которые обучаются. 
# Вызов model.parameters() возвращает итератор по этим параметрам, позволяя обращаться к ним последовательно.
_model_parameters = model.parameters()

# С помощью next(_model_parameters) мы последовательно извлекаем параметры из итератора model.parameters(). 
# Эта структура позволяет отличать центральное слово от контекста, что полезно для обучения таких моделей.
embedding_matrix_center = next(
    _model_parameters
).detach()  # первая матрица из model.parameters(). Она содержит вектора для "центрального слова". Эти вектора используются, когда слово рассматривается как целевое.
embedding_matrix_context = next(
    _model_parameters
).detach()  # вторая матрица из model.parameters(). Она содержит вектора для "контекстного слова". Эти вектора представляют те же слова, но в роли соседних для целевого слова.

# Использование detach() для отсоединения от графа вычислений
# detach() применяется к каждому из этих тензоров (матриц), что выполняет важную роль:

# Вызов .detach() отключает тензор от текущего графа вычислений (в PyTorch, он строится для автоматического дифференцирования).
# Это значит, что мы можем использовать embedding_matrix_center и embedding_matrix_context как статичные тензоры, не влияя на обновление параметров модели и не отслеживая вычисления градиентов для них.
# Отделение тензора от графа вычислений полезно, если мы хотим проанализировать или сохранить текущие веса эмбеддингов как отдельные данные, не изменяя их в ходе дальнейшего обучения.

In [128]:
# Эта функция извлекает векторное представление (эмбеддинг) для заданного слова, используя матрицу эмбеддингов и словарь индексов слов.
def get_word_vector(word, embedding_matrix, word_to_index=word_to_index):
    return embedding_matrix[word_to_index[word]]

Простые проверки:

In [129]:
print("Vector for 'iPhone':", get_word_vector("iphone", embedding_matrix_context))
print("Vector for 'Apple':", get_word_vector("apple", embedding_matrix_context))
print("Vector for 'Dell':", get_word_vector("dell", embedding_matrix_context))


Vector for 'iPhone': tensor([ 0.1171, -0.1187, -0.1331,  0.0011,  0.1511,  0.0262,  0.3928,  0.0176,
        -0.2136, -0.0551,  0.0358, -0.0121, -0.0725, -0.2730,  0.3475, -0.1430,
         0.4183, -0.1028, -0.0006, -0.3810,  0.2902, -0.5154,  0.2898,  0.3048,
        -0.2263, -0.0262,  0.2281,  0.0566,  0.3366,  0.5077,  0.1229,  0.0364])
Vector for 'Apple': tensor([-0.3457, -0.5893, -0.4724,  0.2251,  0.5765,  0.7134, -0.1431, -0.4063,
        -0.2447, -0.0225, -0.0106, -0.3396, -0.1376, -0.0447, -0.1014, -0.5354,
         0.6841, -0.0521, -0.0901, -0.0536,  0.4794, -0.4706,  0.1093, -0.0758,
         0.3781,  0.1222,  0.2602,  0.3493,  0.1864,  0.2476,  0.5047,  0.2276])
Vector for 'Dell': tensor([ 2.0755, -0.4357, -1.1041, -1.3451,  0.4739, -0.4143, -0.5549,  0.0893,
        -0.0075,  0.9684, -0.3060, -0.6028,  0.3119, -0.1141, -0.0195,  0.8174,
         2.4980,  0.3135, -0.4191, -1.5256, -0.0815,  1.6965,  0.0128, -0.9906,
         0.4787,  0.9540, -0.0181, -1.6560, -0.0502, -0.06

In [130]:
# косинусные сходства между векторными представлениями
# F.cosine_similarity(...) вычисляет косинусное сходство между векторами слов. 
# Сходство измеряет, насколько схожи два вектора, и варьируется от -1 до 1, где 1 означает полное совпадение (вектора направлены в одном направлении),
# 0 — отсутствие сходства, а -1 — противоположные направления.


similarity_1 = F.cosine_similarity(
    get_word_vector("iphone", embedding_matrix_context)[None, :],
    get_word_vector("apple", embedding_matrix_context)[None, :],
)
similarity_2 = F.cosine_similarity(
    get_word_vector("iphone", embedding_matrix_context)[None, :],
    get_word_vector("dell", embedding_matrix_context)[None, :],
)
assert similarity_1 > similarity_2

### Зачем это нужно?

**Оценка семантической близости:**

Код помогает проверить, насколько близки значения слов "iphone" и "apple" в векторном пространстве. Это может быть полезно для оценки качества эмбеддингов: если "iphone" и "apple" являются синонимами или имеют схожие значения, их векторы должны быть ближе друг к другу.

**Тестирование модели:**

Это может быть частью процесса тестирования модели, обученной для генерации векторных представлений слов. Если условия не выполняются (т.е. similarity_1 <= similarity_2), это может указывать на проблемы в обучении модели или в самом процессе генерации эмбеддингов.

**Использование в NLP-приложениях:**

Векторные представления слов широко применяются в задачах обработки естественного языка (NLP), таких как классификация текстов, анализ тональности и создание рекомендательных систем. Сравнение сходства помогает находить синонимы, антонимы и связанные слова.

In [131]:
similarity_1 = F.cosine_similarity(
    get_word_vector("windows", embedding_matrix_context)[None, :],
    get_word_vector("laptop", embedding_matrix_context)[None, :],
)
similarity_2 = F.cosine_similarity(
    get_word_vector("windows", embedding_matrix_context)[None, :],
    get_word_vector("macbook", embedding_matrix_context)[None, :],
)
assert similarity_1 > similarity_2

Наконец, взглянем на ближайшие по косинусной мере слова. Функция реализована ниже.

In [132]:
# предназначена для нахождения ближайших слов к заданному слову на основе косинусного сходства векторов их эмбеддингов
# k: Количество ближайших слов, которые нужно вернуть (по умолчанию — 10).
def find_nearest(word, embedding_matrix, word_to_index=word_to_index, k=10):
    word_vector = get_word_vector(word, embedding_matrix)[None, :] # Вызывается функция get_word_vector для получения вектора эмбеддинга заданного слова. 
                                                                   # None, : добавляет новую размерность, превращая вектор в 2D массив (тензор), чтобы его можно было сравнивать с другими векторами.
    dists = F.cosine_similarity(embedding_matrix, word_vector)
    index_sorted = torch.argsort(dists) # сортирует индексы элементов тензора dists в порядке возрастания. 
                                        # Это означает, что на первых позициях будут находиться слова с наименьшим сходством, а на последних — с наибольшим.
    top_k = index_sorted[-k:] # получает последние k индексов из отсортированного массива. Эти индексы соответствуют словам, которые имеют наибольшее косинусное сходство с заданным словом.
    return [(index_to_word[x], dists[x].item()) for x in top_k.numpy()] # Функция возвращает список кортежей, где каждый кортеж состоит из:
                                                                        # слова, соответствующего индексу (из index_to_word), и
                                                                        # значения косинусного сходства (из dists).
                                                                        # Используется top_k.numpy(), чтобы преобразовать индексы в формат NumPy, который более удобен для итерации.

In [133]:
find_nearest("python", embedding_matrix_context, k=10)

[('mira', 0.6053881645202637),
 ('reminiscent', 0.6061620116233826),
 ('timor', 0.607374370098114),
 ('smartly', 0.6189245581626892),
 ('proved', 0.6291970014572144),
 ('prevention', 0.6299614906311035),
 ('cake', 0.6423791646957397),
 ('lightly', 0.6623252630233765),
 ('silent', 0.6768826246261597),
 ('python', 1.0000001192092896)]

Также вы можете визуально проверить, как представлены в латентном пространстве часто встречающиеся слова.

In [134]:
top_k = 5000 # Здесь задаётся переменная top_k, которая определяет количество "топовых" слов, которые мы хотим извлечь, равное 5000.
_top_words = sorted([x for x in word_count_dict.items()], key=lambda x: x[1])[ # Сортируем этот список по частоте (второй элемент каждой пары), то есть от наименьшей частоты к наибольшей.
    -top_k - 100 : -100]  # -top_k - 100 означает, что мы берем элементы, начиная с позиции, которая на 5000+100=5100 от конца (то есть 5100-ное слово от наименьшей частоты) 
                          # и до -100, что означает, что мы исключаем 100 самых частых слов. Таким образом, мы получаем 4900 слов с низкой частотой, исключая 100 наиболее частых слов.
top_words = [x[0] for x in _top_words]
del _top_words

In [135]:
# Здесь используется списковое включение (list comprehension) для создания списка векторов слов.
# top_words — это список слов, которые мы извлекли ранее.
# Для каждого слова x из top_words:
# word_to_index[x]: Используется для получения индекса слова x в словаре word_to_index, который связывает слова с их индексами в матрице векторов.
# embedding_matrix_context[word_to_index[x]]: Получает векторное представление слова из embedding_matrix_context по его индексу.
# [None, :]: Добавляет новую ось в начало вектора, превращая его в 2D массив (т. е. вектор формы (1, embedding_dim)), что позволяет использовать его в операциях, требующих совместимости размерностей.
# torch.cat(..., dim=0): Конкатенирует (объединяет) все векторы в один тензор вдоль первой оси (оси 0).
# В результате получается 2D тензор, в котором каждый ряд соответствует векторному представлению одного слова из списка top_words. 
# Если размерность каждого вектора — embedding_dim, то итоговый тензор будет иметь форму (N, embedding_dim), где N — это количество слов в top_words.
# В конце мы преобразуем получившийся тензор в массив NumPy, чтобы с ним можно было работать в контексте библиотек, которые требуют NumPy-массивы или для дальнейшей обработки.
word_embeddings = torch.cat(
    [embedding_matrix_context[word_to_index[x]][None, :] for x in top_words], dim=0
).numpy()

In [136]:
import bokeh.models as bm # содержит модели, такие как источники данных, инструменты и т.д.
import bokeh.plotting as pl # предоставляет функции для создания визуализаций.
from bokeh.io import output_notebook # позволяет отображать графики непосредственно в Jupyter Notebook.

output_notebook()

# Параметры функции:
# x: данные по оси X (например, координаты точек).
# y: данные по оси Y (например, координаты точек).
# radius: радиус точек на графике (по умолчанию 10).
# alpha: прозрачность точек (по умолчанию 0.25).
# color: цвет точек (по умолчанию "синий"). Если передан как строка, будет использоваться один и тот же цвет для всех точек.
# width и height: размеры графика в пикселях.
# show: флаг, указывающий, нужно ли сразу показывать график (по умолчанию True).
# **kwargs: дополнительные параметры, которые будут добавлены в источник данных и могут быть использованы для отображения дополнительной информации при наведении курсора.
def draw_vectors(
    x,
    y,
    radius=10,
    alpha=0.25,
    color="blue",
    width=600,
    height=400,
    show=True,
    **kwargs,
):
    """draws an interactive plot for data points with auxilirary info on hover"""
    if isinstance(color, str):
        color = [color] * len(x) # Проверяет, является ли color строкой, и если да, то создает список одного цвета, длина которого равна количеству точек x.
    data_source = bm.ColumnDataSource({"x": x, "y": y, "color": color, **kwargs}) # Создает объект ColumnDataSource, который будет использоваться в графике для хранения данных. 
                                                                                  # Это позволяет Bokeh эффективно обновлять график.

    fig = pl.figure(active_scroll="wheel_zoom", width=width, height=height) # Создает новый объект фигуры (графика) с активным инструментом прокрутки (при помощи колеса мыши) и указанными размерами.
    fig.scatter("x", "y", size=radius, color="color", alpha=alpha, source=data_source) # Рисует разбросанные точки на графике, используя данные из data_source. 
                                                                                       # Размер, цвет и прозрачность определяются соответствующими параметрами.

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()])) # Добавляет инструмент наведения (HoverTool), 
                                                                                      # который будет отображать подсказки (tooltips) с дополнительной информацией при наведении курсора на точки. 
                                                                                      # Подсказки будут включать все ключи из kwargs.
    if show:
        pl.show(fig)
    return fig

Loading BokehJS ...

Объект UMAP (Uniform Manifold Approximation and Projection) представляет собой экземпляр класса, который используется для понижения размерности данных с помощью алгоритма UMAP. Давайте разберем, что это такое более подробно.

Что такое UMAP?
UMAP — это алгоритм машинного обучения, предназначенный для уменьшения размерности данных. Он позволяет представлять высокоразмерные данные в более низкоразмерном пространстве (например, в 2D или 3D) с сохранением их структурных и топологических свойств. Это делает его особенно полезным для визуализации и анализа данных.

Основные характеристики UMAP
Топологическая структура:

UMAP стремится сохранить глобальные и локальные структуры данных. Он сохраняет расстояния между точками, так что близкие точки в высокоразмерном пространстве остаются близкими и в пониженном пространстве.
Скорость и масштабируемость:

UMAP более быстр и эффективен по сравнению с другими методами понижения размерности, такими как t-SNE, что делает его подходящим для работы с большими наборами данных.
Настраиваемость:

UMAP предлагает различные параметры, которые можно настраивать, такие как n_neighbors, который определяет количество ближайших соседей, и min_dist, который определяет минимальное расстояние между точками в пониженном пространстве.
Как работает объект UMAP
Когда вы создаете объект UMAP, вы определяете его параметры, а затем можете использовать его для обучения и преобразования данных.

Применение UMAP
Визуализация: Позволяет визуализировать сложные структуры данных, что может помочь в их интерпретации.
Предобработка для других алгоритмов: Пониженные представления могут быть использованы в других алгоритмах машинного обучения (например, для кластеризации).
Изучение семантики: Особенно полезно в NLP (обработке естественного языка) для изучения семантических связей между словами.
Заключение
Таким образом, объект UMAP представляет собой средство для выполнения понижения размерности с использованием алгоритма UMAP. Он позволяет легко настраивать параметры, обучать модель на ваших данных и преобразовывать их в более низкое пространство, сохраняя важные характеристики исходных данных.

In [137]:
# Здесь создается объект UMAP, в котором задается параметр n_neighbors.
# n_neighbors: этот параметр определяет, сколько соседних точек учитывать при построении графа. 
# Уменьшая это значение, вы делаете модель более чувствительной к локальным структурам данных. 
# При значении 5, UMAP будет учитывать 5 ближайших соседей для каждой точки при оценке ее положения в пониженном пространстве.
# .fit_transform(word_embeddings):
# Этот метод одновременно обучает модель UMAP и преобразует входные данные в пониженное пространство.
# word_embeddings: это входные данные (векторы слов), которые вы хотите понизить в размерности. 
# Обычно это многомерные векторы, полученные из предварительно обученных эмбеддингов (например, Word2Vec, GloVe или других моделей).
# embedding: переменная, в которую сохраняются результаты. 
# После выполнения этого кода embedding будет содержать пониженные представления ваших векторов слов в 2D или 3D пространстве (в зависимости от того, как UMAP настроен).

embedding = umap.UMAP(n_neighbors=5).fit_transform(word_embeddings)

# Зачем это нужно?
# Понижение размерности позволяет визуализировать высокоразмерные данные, например, вектора слов, чтобы понять их семантические отношения.
# Это полезно для кластеризации, визуализации и анализа данных, так как вы можете увидеть, какие слова (или их векторы) находятся ближе друг к другу, 
# а какие — дальше, что может помочь в интерпретации семантической близости между словами.



OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [138]:
draw_vectors(embedding[:, 0], embedding[:, 1], token=top_words)

figure(id='p1004', ...)

Для сдачи задания необходимо загрузить функции `subsample_frequent_words` и `get_negative_sampling_prob`, а также сгенерировать файл для посылки ниже и приложить в соответствующую задачу. Успехов!

In [139]:
# do not change the code in the block below
# __________start of block__________
import os
import json

# Этот код проверяет, существует ли файл words_subset.txt в текущем рабочем каталоге. 
# Если файла нет, генерируется ошибка с сообщением о том, что файл необходимо загрузить и разместить в нужной директории.
assert os.path.exists(
    "words_subset.txt"
), "Please, download `words_subset.txt` and place it in the working directory"

# Здесь файл words_subset.txt открывается для чтения. 
# Содержимое файла считывается и разбивается на строки, создавая список selected_words, который содержит слова, перечисленные в файле. 
# Каждое слово будет находиться в отдельном элементе списка.
with open("words_subset.txt") as iofile:
    selected_words = iofile.read().split("\n")

# Функция get_matrix_for_selected_words выполняет следующие действия:
# Входные параметры:
# selected_words: список слов, для которых нужно получить векторы.
# embedding_matrix: матрица векторов слов (размерностью num_words x embedding_dimension).
# word_to_index: словарь, который связывает слова с их индексами в матрице векторов.

# Логика работы:
# Создается пустой список word_vectors для хранения векторов слов.
# Для каждого слова из selected_words:
# Получается его индекс из word_to_index. Если слово не найдено, используется None.
# Инициализируется вектор, состоящий из нулей, длина которого равна размерности векторов в embedding_matrix.
# Если индекс слова найден (не None), то вектор обновляется значениями из embedding_matrix для этого слова, и он преобразуется в список с помощью numpy().tolist().
# Вектор добавляется в список word_vectors.
# В конце функция возвращает список векторов word_vectors.
def get_matrix_for_selected_words(selected_words, embedding_matrix, word_to_index):
    word_vectors = []
    for word in selected_words:
        index = word_to_index.get(word, None)
        vector = [0.0] * embedding_matrix.shape[1]
        if index is not None:
            vector = embedding_matrix[index].numpy().tolist()
        word_vectors.append(vector)
    return word_vectors


word_vectors = get_matrix_for_selected_words(
    selected_words, embedding_matrix_context, word_to_index
)

with open("submission_dict.json", "w") as iofile:
    json.dump(word_vectors, iofile)
print("File saved to `submission_dict.json`")
# __________end of block__________

File saved to `submission_dict.json`
